In [ ]:
save_name = 'CustomPneumonia_Oversample_L2_Sum_PosWeight_12_Targets'

In [ ]:
import sys
import os, os.path

sys.path.append(os.path.join(os.getcwd() ,'/modules'))
root_path = "C:/git/Springboard-Public/Capstone Project 2/"
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    root_path = "/content/drive/My Drive/Capstone Project 2/"

print('Current Working Dir: ', os.getcwd())
print('Root Path: ', root_path)

# We need to set the working directory since we are using relative paths from various locations
if os.getcwd() != root_path:
  os.chdir(root_path)

In [ ]:
from modules.lib.StandardTraining import *
from modules.models.CustomPneumonia import CustomPneumoniaNN

from torchsummary import summary

# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

%matplotlib inline

In [ ]:
force_cpu = True
device = torch.device('cuda' if ~force_cpu and torch.cuda.is_available() else 'cpu')
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print(f'Working on device={device}')


In [ ]:
out_channels=12
net_kwargs = {'out_channels':out_channels, 'image_size':(224,224)}
net = CustomPneumoniaNN(**net_kwargs)

net = nn.DataParallel(net)
net.to(device)

summary(net, (1, 224, 224))

In [ ]:
standardTraining = StandardTraining(number_images=0, 
                                     batch_size=8, 
                                     learning_rate=1e-5, 
                                     num_epochs=1,
                                     device=device, 
                                     net=net,
                                     image_width = 224, #Model more memory consuming so pare down size
                                     image_height = 224,
                                     use_positivity_weights=True,
                                     observation_min_count=150,
                                     loss_reduction='sum',
                                     l2_reg=1e-1,
                                     augPercent=.4,
                                     target_columns=training_columns,
                                     save_path= f'notebooks/ModelRuns/saved/{save_name}/',
                                     net_name = 'CustomPneumoniaNN',
                                     net_kwargs = net_kwargs)

In [ ]:
standardTraining.train()

In [ ]:
standardTraining.displayMetrics()

In [ ]:
standardTraining.save()